# BASICS 03 - HOTS in PyTorch

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ../hots
import tonic, torch, os
from timesurface import timesurface
from network import network
from utils import get_loader

print(f'Tonic version installed -> {tonic.__version__}')

print(f'Number of GPU devices available: {torch.cuda.device_count()}')
for N_gpu in range(torch.cuda.device_count()):
    print(f'GPU {N_gpu+1} named {torch.cuda.get_device_name(N_gpu)}')

/home/antoine/homhots/hotsline/hots
Tonic version installed -> 1.0.15
Number of GPU devices available: 1
GPU 1 named GeForce RTX 2080 Ti


## Initialization of the network

In [ ]:
name = 'homeohots'
homeo = True
timestr = '2022-04-01'
dataset_name = 'nmnist'

Rz = (2, 4, 8)
N_neuronz = (4, 8, 16)
tauz = (5e3, 5e4, 5e5)

network = network(name, dataset_name, timestr, dataset.sensor_size, nb_neurons = N_neuronz, tau = tauz, R = Rz, homeo = homeo)

## Loading of the dataset for the clustering phase

In [8]:
trainset = True
transform = tonic.transforms.NumpyAsType(int)
dataset = tonic.datasets.NMNIST(save_to='../../Data/', train=trainset, transform=transform)
loader = get_loader(dataset, kfold = 300, shuffle=True)
#get_dataset_info(dataset, properties = ['time', 'mean_isi', 'nb_events']);
print(f'number of samples in the dataset: {len(loader)}')

number of samples in the dataset: 200


## Unsupervised clustering

In [6]:
if not os.path.exists('../Records/'):
    os.mkdir('../Records/')
    os.mkdir('../Records/networks/')
filtering_threshold = [2*Rz[L] for L in range(len(Rz))]
path = '../Records/networks/'+network.name+'.pkl'
if not os.path.exists(path):
    network.clustering(loader, dataset.ordering, filtering_threshold)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [18:25<00:00,  5.53s/it]


## Coding of the training set

In [ ]:
loader = get_loader(dataset, kfold = None, shuffle=True)
network.coding(loader, dataset.ordering, dataset.classes, filtering_threshold, training=trainset)

  4%|████████▉                                                                                                                                                                                                                        | 2374/60000 [34:20<18:00:57,  1.13s/it]

## Coding of the testing set

In [ ]:
trainset = False
dataset = tonic.datasets.NMNIST(save_to='../../Data/', train=trainset, transform=transform)
loader = get_loader(dataset, kfold = 300, shuffle=True)
network.coding(loader, dataset.ordering, dataset.classes, filtering_threshold, training=trainset)

## TODO: 
- add plotlayer from neuronhots
- add plot of the loss
- add the MLR